In [1]:
from IPython import display
import os

In [2]:
!pip install git+https://github.com/facebookresearch/nougat
display.clear_output()

In [3]:
!curl -o quantum_physics.pdf https://www.sydney.edu.au/science/chemistry/~mjtj/CHEM3117/Resources/postulates.pdf


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69838  100 69838    0     0   106k      0 --:--:-- --:--:-- --:--:--  106k


In [4]:
!nougat --markdown pdf '/content/quantum_physics.pdf' --out 'physics'

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.11/dist-packages/nougat/transforms.py:146: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  alb.ElasticTransform(
downloading nougat checkpoint version 0.1.0-small to path /root/.cache/torch/hub/nougat-0.1.0-small
config.json: 100% 557/557 [00:00<00:00, 3.12Mb/s]
pytorch_model.bin: 100% 956M/956M [00:19<00:00, 52.6Mb/s]
special_tokens_map.json: 100% 96.0/96.0 [00:00<00:00, 429kb/s]
tokenizer.json: 100% 2.04M/2.04M [00:00<00:00, 58.1Mb/s]
tokenizer_config.json: 100% 106/106 [00:00<00:00, 491kb/s]
INFO:root:Output directory does not exist. Creating output directory.
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539

In [5]:
display.Latex('/content/physics/quantum_physics.mmd')

<IPython.core.display.Latex object>

In [8]:
!curl -o test.pdf https://arxiv.org/pdf/2504.07652

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  321k  100  321k    0     0  1668k      0 --:--:-- --:--:-- --:--:-- 1676k


In [13]:
!nougat --markdown pdf '/content/test.pdf' --out 'test'

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.11/dist-packages/nougat/transforms.py:146: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  alb.ElasticTransform(
INFO:root:Output directory does not exist. Creating output directory.
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/nougat/model.py:437: UserWarning: var(): degrees of freedom is <= 0

In [15]:
display.Latex('/content/test/test.mmd')

<IPython.core.display.Latex object>

In [16]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

image = Image.open("test.jpg")
inputs = processor(image, return_tensors="pt")

out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)
print(caption)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a graph with a line graph and a line graph


In [10]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoProcessor, AutoModelForCausalLM
from PIL import Image
from torch.cuda.amp import autocast


def generate_caption_with_blip(image_path, model_size="base"):
    """
    Genera una descrizione dettagliata dell'immagine usando BLIP.

    Args:
        image_path: Percorso all'immagine
        model_size: Dimensione del modello ("base" o "large")

    Returns:
        Stringa contenente la descrizione dell'immagine
    """
    # Seleziona il modello in base alla dimensione richiesta
    if model_size == "large":
        model_name = "Salesforce/blip-image-captioning-large"
    else:
        model_name = "Salesforce/blip-image-captioning-base"

    # Carica modello e processore
    processor = BlipProcessor.from_pretrained(model_name)
    model = BlipForConditionalGeneration.from_pretrained(model_name)

    # Configura per usare GPU se disponibile
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()
    # Carica e pre-processa l'immagine
    image = Image.open(image_path)

    # Genera la didascalia in due modi

    # Use torch.no_grad to avoid computing gradients
    with torch.no_grad():
        # Process the image and generate captions in one pass
        inputs = processor(image, return_tensors="pt").to(device)

        # Conditional caption
        conditional_inputs = processor(image, text="Describe the imag", return_tensors="pt").to(device)
        with autocast():
            conditional_outputs = model.generate(**conditional_inputs)
        conditional_caption = processor.decode(conditional_outputs[0], skip_special_tokens=True)

    return {
        #"basic_caption": caption,
        "detailed_caption": conditional_caption
    }

In [12]:
results = generate_caption_with_blip("test.jpg")

print("\nRISULTATI:")
#print("\n1. BLIP (base):")
#print(results["blip_basic"])

print("\n2. BLIP (dettagliato):")
print(results["blip_detailed"])

#print("\n3. GIT:")
#print(results["git"])
"""
for key, value in results.items():
    print(f"=== {key.upper()} ===\n")
    print(value)
    print("\n\n")
"""

<ipython-input-10-a34ad2dacf83>:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



RISULTATI:

2. BLIP (dettagliato):
describe the imag model for training and development


'\nfor key, value in results.items():\n    print(f"=== {key.upper()} ===\n")\n    print(value)\n    print("\n\n")\n'

In [1]:
!pip install --upgrade transformers

In [2]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2Model
import torch
# Load LLaVA model
"""
model_id = "llava-hf/llava-1.5-13b-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load and process image
image = Image.open("image.jpg").convert('RGB')
prompt = "Describe this image with as much detail as possible."
inputs = processor(prompt, image, return_tensors="pt").to(device)

# Generate description
outputs = model.generate(**inputs, max_length=300, do_sample=True, temperature=0.7)
description = processor.decode(outputs[0], skip_special_tokens=True)
print("### LLAVA ###")
print(description)

"""
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2Model.from_pretrained("Salesforce/blip2-opt-2.7b")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

image = Image.open("test.jpg")

# For rich, detailed captions
inputs = processor(image, "Describe this image in detail", return_tensors="pt").to(device)
outputs = model(**inputs)
print("BLIP2")
print(f"Detailed caption: {outputs}")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 4.12 MiB is free. Process 132206 has 14.73 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 77.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)